In [1]:
import pandas as pd
from bashlint import data_tools
import pickle

Setting bashlex grammar using file: c:\Users\madhu\Desktop\TSE Project\bashlint\grammar\grammar100.txt
Bashlint grammar set up (124 utilities)



In [2]:
final = pickle.load(open('Final_with_multiple_codes.txt', 'rb'))

In [3]:
type(final)

pandas.core.frame.DataFrame

In [4]:
final.shape

(31505, 3)

In [5]:
final.head(3)

,Natural_language_description,Code1,Code2
0,use find xargs make symlinks selected files ke...,"[mkdir -p {dirname}, # usage mklinks from to [...",None
1,using regular expression rm ls,[find . -maxdepth 1 -regextype posix-extended ...,[rm file[1-9][1-9][1-9].txt file[1-9][1-9].txt\n]
2,pass list file names xargs checking file exists,"[command1 |\nxargs sh -c 'for p do [ -f ""$p"" ]...",[existing_plain_readable_files_only() {\n per...


In [6]:
#TAKING A SAMPLE CODE TO TEST THE PARSER

sample_code = final.Code1.values[1]

In [7]:
sample_code

["find . -maxdepth 1 -regextype posix-extended -regex '.*/file[1-9]{2,3}.txt' -delete\n"]

In [8]:
type(sample_code)

list

In [9]:
#PARSING USING BASHLINT

cmd_ast = data_tools.bash_parser(sample_code[0])

data_tools.pretty_print(cmd_ast)

ROOT()
    UTILITY(find)
        ARGUMENT(.)<Path>
        FLAG(-maxdepth)
            ARGUMENT(1)<Quantity>
        FLAG(-regextype)
            ARGUMENT(posix-extended)<Type>
        FLAG(-regex)
            ARGUMENT('.*/file[1-9]{2,3}.txt')<Regex>
        FLAG(-delete)


In [10]:
#RETURNS ALL THE UTILITIES IN THE CODE

data_tools.get_utilities(cmd_ast)

{'find'}

In [11]:
data_tools.ast2tokens(cmd_ast)

['find',
 '.',
 '-maxdepth',
 '1',
 '-regextype',
 'posix-extended',
 '-regex',
 "'.*/file[1-9]{2,3}.txt'",
 '-delete']

In [12]:
#CLEANS THE COMMAND TO CREATE A TEMPLATE

data_tools.ast2template(cmd_ast)

'find Path -maxdepth Quantity -regextype posix-extended -regex Regex -delete'

In [13]:
len(final.Code1.values)

31505

In [14]:
indexes = []

In [15]:
#CLEANING THE CODES AND CONVERTING TO A TEMPLATE

for i in range(len(final.Code1.values)):
    if(final.Code1.values[i]!='None'):
        l=[]
        for j in range(len(final.Code1.values[i])):
            try:
                cmd = final.Code1.values[i][j]
                template = data_tools.cmd2template(cmd)
                if(len(template)>0):
                    indexes.append(i)
                    l.append(template)
            except:
                continue
        if(len(l)==0):
            final.Code1.values[i]='None'
        else:
            final.Code1.values[i]=l


In [16]:
c=0
for i in range(len(final)):
    if(final.Code1.values[i]==final.Code2.values[i]=='None'):
        c=c+1
c

9856

In [17]:
#CLEANING THE CODES AND CONVERTING TO A TEMPLATE

for i in range(len(final.Code2.values)):
    if(final.Code2.values[i]!='None'):
        l=[]
        for j in range(len(final.Code2.values[i])):
            try:
                cmd = final.Code2.values[i][j]
                template = data_tools.cmd2template(cmd)
                if(len(template)>0):
                    indexes.append(i)
                    l.append(template)
            except:
                continue
        if(len(l)==0):
            final.Code2.values[i]='None'
        else:
            final.Code2.values[i]=l


In [18]:
c=0
for i in range(len(final)):
    if(final.Code1.values[i]==final.Code2.values[i]=='None'):
        c=c+1
c

15734

In [19]:
indexes=list(set(indexes))

In [20]:
len(indexes)

15771

In [21]:
#REMOVING THE ONES WHICH CONTAINS MULTIPLE LINE CODES AND OTHER ELEMENTS WHICH THE PARSER CANNOT PARSE
#TO ENSURE CLEANLINESS

final=final.iloc[indexes]

In [22]:
c=0
for i in range(len(final)):
    if(final.Code1.values[i]==final.Code2.values[i]=='None'):
        c=c+1
c

0

In [23]:
len(final)

15771

In [24]:
final=final.reset_index(drop=True)

In [25]:
len(final)

15771

In [27]:
final

,Natural_language_description,Code1,Code2
0,use find xargs make symlinks selected files ke...,[mkdir -p Directory],None
1,using regular expression rm ls,[find Path -maxdepth Quantity -regextype posix...,[rm File File]
2,open multiple files find output,[shopt -s globstar],None
3,remove files provided pipe,"[xargs -I {} rm {}, xargs -d Regex -I {} rm -f...",None
4,search word entire content directory linux,"[grep -r Regex File, find Path -type f -exec g...","[grep -R Regex File, ls -l -R | grep Regex]"
...,...,...,...
15766,backspace insert mode vi erase character,None,[set Regex Regex Regex Regex]
15767,show user ubuntu login screen,None,"[cp File File, nano File]"
15768,enable user jobs upstart,None,[vim File]
15769,prevent new user anything dangerous,None,"[chmod Permission File, man Regex]"


In [28]:
#DATASET WITH POSSIBLY MULTIPLE CODES PER EXAMPLE(NOISY)

pickle.dump(final, open('Dataset_multiple.csv', 'wb'))

In [29]:
#FINDING EXAMPLES WHICH WILL GIVE US SINGLE CODES


index_sing=[]
for i in range(len(final)):
    if((final.Code1.values[i]!=None and len(final.Code1.values[i])==1) or (final.Code2.values[i]!=None and len(final.Code2.values[i])==1)):
        index_sing.append(i)

In [30]:
len(index_sing)

10260

In [31]:
final=final.iloc[index_sing]

In [32]:
final=final.reset_index(drop=None)

In [33]:
final

,index,Natural_language_description,Code1,Code2
0,0,use find xargs make symlinks selected files ke...,[mkdir -p Directory],None
1,1,using regular expression rm ls,[find Path -maxdepth Quantity -regextype posix...,[rm File File]
2,2,open multiple files find output,[shopt -s globstar],None
3,5,create md hash recursive file listing paths sp...,[find Path -type f -exec md5 {} +],None
4,7,delete files filtered awk,"[ls -l | awk Program | xargs -I {} rm {}, find...",[find Path -name Regex -or -type d -prune -or ...
...,...,...,...,...
10255,15761,criteria user background become login backgrou...,None,[chmod -R Permission File]
10256,15764,set new xdg open setting,None,[grep Regex -l -r File File File]
10257,15766,backspace insert mode vi erase character,None,[set Regex Regex Regex Regex]
10258,15768,enable user jobs upstart,None,[vim File]


In [34]:
for i in range(len(final)):
    if(len(final.Code1.values[i])>1):
        final.Code1.values[i]='None'
    if(len(final.Code2.values[i])>1):
        final.Code2.values[i]='None'
    

In [35]:
c=0
for i in range(len(final)):
    if(final.Code1.values[i]==final.Code2.values[i]=='None'):
        c=c+1
c

0

In [36]:
final

,index,Natural_language_description,Code1,Code2
0,0,use find xargs make symlinks selected files ke...,[mkdir -p Directory],None
1,1,using regular expression rm ls,[find Path -maxdepth Quantity -regextype posix...,[rm File File]
2,2,open multiple files find output,[shopt -s globstar],None
3,5,create md hash recursive file listing paths sp...,[find Path -type f -exec md5 {} +],None
4,7,delete files filtered awk,None,[find Path -name Regex -or -type d -prune -or ...
...,...,...,...,...
10255,15761,criteria user background become login backgrou...,None,[chmod -R Permission File]
10256,15764,set new xdg open setting,None,[grep Regex -l -r File File File]
10257,15766,backspace insert mode vi erase character,None,[set Regex Regex Regex Regex]
10258,15768,enable user jobs upstart,None,[vim File]


In [37]:
#CLEAN DATA

final_2 = {'Natural_language_description':[], 'Code':[]}

In [38]:
for i in range(len(final)):
    final_2['Natural_language_description'].append(final.Natural_language_description.values[i])
    if(final.Code1.values[i]!='None'):
        final_2['Code'].append(final.Code1.values[i][0])
    else:
        final_2['Code'].append(final.Code2.values[i][0])

In [39]:
final_2=pd.DataFrame.from_dict(final_2)

In [40]:
c=0
for i in range(len(final_2)):
    if(final_2.Code.values[i]=='None'):
        c=c+1
c

0

In [41]:
final_2

,Natural_language_description,Code
0,use find xargs make symlinks selected files ke...,mkdir -p Directory
1,using regular expression rm ls,find Path -maxdepth Quantity -regextype posix-...
2,open multiple files find output,shopt -s globstar
3,create md hash recursive file listing paths sp...,find Path -type f -exec md5 {} +
4,delete files filtered awk,find Path -name Regex -or -type d -prune -or -...
...,...,...
10255,criteria user background become login backgrou...,chmod -R Permission File
10256,set new xdg open setting,grep Regex -l -r File File File
10257,backspace insert mode vi erase character,set Regex Regex Regex Regex
10258,enable user jobs upstart,vim File


In [42]:
pickle.dump(final_2, open('Dataset.csv', 'wb'))